In [7]:
import csv
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [8]:
df = pd.read_csv('ddv_link.csv')
links = df['link'].values.tolist()

In [9]:
browser = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2} # Tắt thông báo Chrome
browser.add_experimental_option("prefs",prefs)
browser = webdriver.Chrome(chrome_options=browser)

C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [10]:
data = {'Tên': [], 'Link': [], 'Giá': [], 'Ảnh': [], 'Màu': [], 'Màn hình': [], 'Camera sau': [], 'Camera trước': [], 'Hệ điều hành - CPU': [], 'Bộ nhớ - Lưu trữ': [], 'Kết nối': [], 'Thông tin pin - Sạc': []}
detail = list(data.keys())
detail = detail[5:13]

In [11]:
def extract(link):
    browser.get(link)

    # Link
    data['Link'].append(link)

    # Name
    name = browser.find_element_by_xpath('//div[@class="page-title-wrapper product"]/h1/span').text
    data['Tên'].append(name)

    # Price
    price = browser.find_element_by_class_name('price').text.replace('.', '').replace('₫', '').replace(' ', '')
    data['Giá'].append(price)

    # Color
    color = []
    colors = browser.find_elements_by_xpath('//div[@class="  control-option"]/*/span[1]')
    for c in colors:
        color.append(c.text)
    color = ', '.join(str(i) for i in color)
    data['Màu'].append(color)
    print(data['Màu'])

    # Image
    img = browser.find_element_by_xpath('//div[@class="nav-gallery-right"]/img').get_attribute('src')
    data['Ảnh'].append(img)

    # Detail
    att_table = browser.find_element_by_xpath('//p[@class="showmoreattribute"]/a')
    browser.execute_script("arguments[0].click();", att_table)

    att = []
    key = browser.find_elements_by_xpath('//div[@class="modal-attribute"]/div//div[2]/div/div/*/div[1]')
    value = browser.find_elements_by_xpath('//div[@class="modal-attribute"]/div//div[2]/div/div/*/div[2]')
    for i in range(len(key)):
        k = key[i].text
        att.append(k)
        if k in detail:
            v = value[i].text.replace('\n', '')
            data[k].append(v)
    for i in range(len(detail)):
        if detail[i] not in att:
            data[detail[i]].append('')

    return data


In [12]:
for i in range(len(links)):
    data = extract(links[i])

['Màu xanh, Màu xám']
['Màu xanh, Màu xám', 'Màu xanh, Màu đen, Màu tím']
['Màu xanh, Màu xám', 'Màu xanh, Màu đen, Màu tím', 'Màu xanh, Màu đen, Màu tím']
['Màu xanh, Màu xám', 'Màu xanh, Màu đen, Màu tím', 'Màu xanh, Màu đen, Màu tím', 'Màu xanh, Màu đen, Màu tím']


WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.93)


In [11]:
browser.quit()

In [10]:
df = pd.DataFrame.from_dict(data)

In [11]:
df.to_csv('ddv.csv', index = False)